# Titanic

## Librerie

In [188]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sl.__version__)

2.2.0
1.0.4
1.18.4
0.22.2.post1


## Download dei Dataset

In [0]:
url_train = 'https://raw.githubusercontent.com/rirolli/Titanic/master/train.csv'
url_test = 'https://raw.githubusercontent.com/rirolli/Titanic/master/test.csv'

titanic_load_train = pd.read_csv(url_train)
titanic_load_test = pd.read_csv(url_test)

In [190]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Ottimizzazione del Dataset

In [0]:
# titanic_load_train = titanic_load_train.fillna(np.nan)
for elem_train in titanic_load_train:
  titanic_load_train[elem_train] = (pd.Categorical(titanic_load_train[elem_train])).codes

# titanic_load_test = titanic_load_test.fillna(np.nan)
for elem_test in titanic_load_test:
  titanic_load_test[elem_test] = (pd.Categorical(titanic_load_test[elem_test])).codes


In [192]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,2,108,1,28,1,0,523,18,-1,2
1,1,1,0,190,0,51,1,0,596,207,81,0
2,2,1,2,353,0,34,0,0,669,41,-1,2
3,3,1,0,272,0,47,1,0,49,189,55,2
4,4,0,2,15,1,47,0,0,472,43,-1,2


In [193]:
titanic_load_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,206,1,44,0,0,152,24,-1,1
1,1,2,403,0,60,1,0,221,5,-1,2
2,2,1,269,1,74,0,0,73,41,-1,1
3,3,2,408,1,34,0,0,147,34,-1,2
4,4,2,178,0,27,1,1,138,46,-1,2


### Ripartizione del Dataset

In [0]:
# Dividiamo il train set in due parti così da poter addestrare la rete:
# y che contiene solo la colonna 'Survived' che consiste nella soluzione al problema
# X che contiene tutte le altre colonne.
y = titanic_load_train.Survived
X = titanic_load_train.drop(labels=['Survived'], axis=1)

# I Dati vengono, a loro volta, divisi in altre due parti: una per
# l'addestramento e una per la valutazione della rete.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)

# I dati vengono poi memorizzati dentro un Dataset di Tensorflow per sfruttarli
# per l'addestramento. Questa stessa operazione viene poi fatta anche per il
# test set e per l'evaluetion set.
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))).shuffle(len(X_train)).batch(1)
val_dataset = (tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))).shuffle(len(X_val)).batch(1)
test_dataset = (tf.data.Dataset.from_tensor_slices(titanic_load_test.values)).batch(1)

## La rete neurale

In [0]:
num_inputs = X_train.shape[1]

In [196]:
model = keras.Sequential([
  tf.keras.layers.Dense(24, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 24)                288       
_________________________________________________________________
dense_34 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 25        
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


## Addestramento della rete

In [197]:
# Addestramento della rete neurale
model.fit(train_dataset, epochs=500)

Epoch 1/500
712/712 [==============================] - 1s 2ms/step - loss: 5.7509 - accuracy: 0.5492
Epoch 2/500
712/712 [==============================] - 1s 1ms/step - loss: 2.8335 - accuracy: 0.5941
Epoch 3/500
712/712 [==============================] - 1s 1ms/step - loss: 2.3826 - accuracy: 0.6334
Epoch 4/500
712/712 [==============================] - 1s 1ms/step - loss: 2.2342 - accuracy: 0.6250
Epoch 5/500
712/712 [==============================] - 1s 1ms/step - loss: 1.6431 - accuracy: 0.6306
Epoch 6/500
712/712 [==============================] - 1s 2ms/step - loss: 1.5143 - accuracy: 0.6475
Epoch 7/500
712/712 [==============================] - 1s 2ms/step - loss: 1.3333 - accuracy: 0.6573
Epoch 8/500
712/712 [==============================] - 1s 2ms/step - loss: 1.4416 - accuracy: 0.6419
Epoch 9/500
712/712 [==============================] - 1s 2ms/step - loss: 1.2643 - accuracy: 0.6348
Epoch 10/500
712/712 [==============================] - 1s 2ms/step - loss: 1.1731 - accura

## Valutazione della rete

In [198]:
# Valutazione del modello
val_loss, val_acc = model.evaluate(val_dataset)
print("\nTest accuracy: {:.2f} ({:.2%})".format(val_acc, val_acc))

179/179 [==============================] - 0s 1ms/step - loss: 1.2479 - accuracy: 0.7765

Test accuracy: 0.78 (77.65%)


## Previsione

In [0]:
predictions = model.predict(test_dataset)

In [204]:
for prediction in predictions:
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]))

Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.33%
Predicted survival: 1.09%
Predicted survival: 0.10%
Predicted survival: 100.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 100.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 31.32%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 99.54%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.04%
Predicted survival: 0.00%
Predicted survival: 0.43%
Predicted survival: 37.10%
Predicted survival: 0.00%
Predicted survival: 100.00%
Predicted survival: 0.00%
Predicted survival: 19.05%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 0.01%
Predicted survival: 0.45%
Predicted survival: 0.00%
Predicted survival: 0.00%
Predicted survival: 73.73%
Predicted survival: 48.55%
